In [1]:
# imports necessários:
import pandas as pd
from pathlib import PurePath, PurePosixPath, PosixPath 
from datetime import datetime
import sqlite3 as sql


## Coleta de dados site IBGE

In [2]:
#Coletando dados dos municípios de Minas Gerais
url = 'https://www.ibge.gov.br/explica/codigos-dos-municipios.php#MG'

ibge_dados_mg = pd.DataFrame(pd.read_html(url, match='Municípios de Minas Gerais')[0])

#Ajustando a nomenclatura das colunas do Data Frame:
ibge_dados_mg = ibge_dados_mg.rename(columns = {'Municípios de Minas Gerais': 'nomeMunic', 'Códigos': 'codMunic'})

#Adicionado coluna ao Data Frame:
ibge_dados_mg['dtCarga'] = datetime.today().strftime('%d/%m/%Y %H:%M')

#Ajustando index do Data Frame, nome enumeração do index
ibge_dados_mg.index.name = 'idMunic'
ibge_dados_mg.index = ibge_dados_mg.index + 1


## Configuração Banco de Dados: ODS e DW - SQLite

In [3]:
##Manipulando sistema de arquivos:
endereco = PosixPath('/home/juvito/Projeto-pyton')

bd_ods = endereco / "ibgeODS.db"
bd_dw = endereco / "ibgeDW.db"

if endereco.exists():
    if (bd_ods.exists() and bd_dw.exists()):
        print('Bancos de dados já existem!')
    else:
        bd_ods.touch()
        bd_dw.touch()
        print('Bancos de dados criados!')
else:
    print('Endereço não existe, favor verificar!')
    

Bancos de dados já existem!


## Manipulando Banco de Dados SQLite

In [4]:
#Conectar no ODS
conexao_ods = sql.connect(bd_ods)

#Criar e/ou popular tabela tbLogMunic com o DF ibge_dados_mg
ibge_dados_mg.to_sql('tbLogMunic', conexao_ods, if_exists='append')

conexao_ods.commit()
conexao_ods.close()

print('Carga ODS concluída')

Carga ODS concluída


In [5]:
#Conectar no DW
conexao_dw = sql.connect(bd_dw)

#Criar Data Frame para popular o DW
ibge_dados_mg_dw = ibge_dados_mg[['codMunic','nomeMunic']]

#Criar e/ou popular a dimensão dMunicipio com o DF ibge_dados_mg_dw
ibge_dados_mg_dw.to_sql('dMunicipio', conexao_dw, if_exists='replace')

conexao_dw.commit()
conexao_dw.close()

print('Carga DW concluída')

Carga DW concluída
